In [1]:
import os, s3fs, json
import xarray as xr

from kerchunk.combine import MultiZarrToZarr

In [2]:
fs = s3fs.S3FileSystem()
date = '20230312'

kerchunk_url = f's3://azavea-noaa-hydro-data/noaa-nwm-pds/nwm.{date}/short_range/nwm.t*z.short_range.channel_rt.*.conus.json'
kerchunk_refs = fs.glob(kerchunk_url)

In [3]:
%%time

kerchunks = []

for ref in kerchunk_refs:
    with fs.open(ref, mode='r') as f:
        kerchunks.append(json.load(f))

CPU times: user 1.75 s, sys: 93.6 ms, total: 1.84 s
Wall time: 16.1 s


In [4]:
%%time

mzz = MultiZarrToZarr(
    kerchunks,
    remote_protocol='s3', remote_options={'anon': True},
    concat_dims=['reference_time', 'time'])

combined = mzz.translate()

CPU times: user 3.49 s, sys: 561 ms, total: 4.05 s
Wall time: 3.61 s


In [5]:
%%time

combined_url = f's3://azavea-noaa-hydro-data/noaa-nwm-pds/nwm.20230312/short_range/nwm.short_range.channel_rt.conus.json'

with fs.open(combined_url, mode='wb') as outfile:
    outfile.write(json.dumps(combined).encode())

CPU times: user 12.8 ms, sys: 26 µs, total: 12.8 ms
Wall time: 96.8 ms
